# Выбор локации для скважины

Допустим, мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
try:
    df0 = pd.read_csv('/datasets/geo_data_0.csv')
    df1 = pd.read_csv('/datasets/geo_data_1.csv')
    df2 = pd.read_csv('/datasets/geo_data_2.csv')
        
except:
    df0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
    df1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
    df2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')


In [3]:
display(
    df0.head(),
    df0.info(),
    df0.describe(include='all'),
    df0.corr())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


C:\Users\lyser\AppData\Local\Temp\ipykernel_21016\3758813567.py:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df0.corr())


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


None

,id,f0,f1,f2,product
count,100000,100000.000000,100000.000000,100000.000000,100000.000000
unique,99990,NaN,NaN,NaN,NaN
top,fiKDv,NaN,NaN,NaN,NaN
freq,2,NaN,NaN,NaN,NaN
mean,NaN,0.500419,0.250143,2.502647,92.500000
std,NaN,0.871832,0.504433,3.248248,44.288691
min,NaN,-1.408605,-0.848218,-12.088328,0.000000
25%,NaN,-0.072580,-0.200881,0.287748,56.497507
50%,NaN,0.502360,0.250252,2.515969,91.849972
75%,NaN,1.073581,0.700646,4.715088,128.564089


,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [4]:
display(
    df1.head(),
    df1.info(),
    df1.describe(include='all'),
    df1.corr())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


C:\Users\lyser\AppData\Local\Temp\ipykernel_21016\1628433426.py:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df1.corr())


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


None

,id,f0,f1,f2,product
count,100000,100000.000000,100000.000000,100000.000000,100000.000000
unique,99996,NaN,NaN,NaN,NaN
top,wt4Uk,NaN,NaN,NaN,NaN
freq,2,NaN,NaN,NaN,NaN
mean,NaN,1.141296,-4.796579,2.494541,68.825000
std,NaN,8.965932,5.119872,1.703572,45.944423
min,NaN,-31.609576,-26.358598,-0.018144,0.000000
25%,NaN,-6.298551,-8.267985,1.000021,26.953261
50%,NaN,1.153055,-4.813172,2.011479,57.085625
75%,NaN,8.621015,-1.332816,3.999904,107.813044


,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [5]:
display(
    df2.head(),
    df2.info(),
    df2.describe(include='all'),
    df2.corr())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


C:\Users\lyser\AppData\Local\Temp\ipykernel_21016\2443682788.py:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df2.corr())


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


None

,id,f0,f1,f2,product
count,100000,100000.000000,100000.000000,100000.000000,100000.000000
unique,99996,NaN,NaN,NaN,NaN
top,VF7Jo,NaN,NaN,NaN,NaN
freq,2,NaN,NaN,NaN,NaN
mean,NaN,0.002023,-0.002081,2.495128,95.000000
std,NaN,1.732045,1.730417,3.473445,44.749921
min,NaN,-8.760004,-7.084020,-11.970335,0.000000
25%,NaN,-1.162288,-1.174820,0.130359,59.450441
50%,NaN,0.009424,-0.009482,2.484236,94.925613
75%,NaN,1.158535,1.163678,4.858794,130.595027


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


### Вывод

Начальные данные выглядят достаточно качественно: нет ни пропусков, ни дубликатов. Типы данных не требуют изменения.  
Есть несколько повторов в столбцах id, но это не дубликаты.  
Заметна корреляция (Пирсона), которая говорит о наличии линейной связи между значениями переменных f2 и product.  
Можно проверить переменные на выбросы, но не имея представления о назначении переменных, мы не сможем сделать никаких выводов о выбросах. Выбросы могут оказать влияние на методы оценки качества модели (RMSE и MAE).  
Из всего этого трудно сделать какие-то выводы, потому что у нас недостаточно информации о переменных.

## Разбиваем данные на обучающую и валидационную выборки

Удалим столбец id, он бесполезен для дальнейшего исследования

In [6]:
features_0 = df0.drop(['id', 'product'], axis=1)
target_0 = df0['product']

features_1 = df1.drop(['id', 'product'], axis=1)
target_1 = df1['product']

features_2 = df2.drop(['id', 'product'], axis=1)
target_2 = df2['product']

In [7]:
state = np.random.RandomState(12345)

(features_trn_0, features_vld_0, target_trn_0, target_vld_0) = train_test_split(
    features_0,
    target_0,
    test_size=0.25,
    random_state=state)

(features_trn_1, features_vld_1, target_trn_1, target_vld_1) = train_test_split(
    features_1,
    target_1,
    test_size=0.25,
    random_state=state)

(features_trn_2, features_vld_2, target_trn_2, target_vld_2) = train_test_split(
    features_2,
    target_2,
    test_size=0.25,
    random_state=state)

In [8]:
features_trn_0.shape, features_vld_0.shape, target_trn_0.shape, target_vld_0.shape

((75000, 3), (25000, 3), (75000,), (25000,))

### Вывод

Разбили правильно

## Обучение и проверка модели

По условия для обучения модели можно использовать только модель Линейной регрессии

In [9]:
model_0 = LinearRegression()
model_0.fit(features_trn_0, target_trn_0)

predictions_vld_0 = model_0.predict(features_vld_0)
predictions_vld_0 = pd.Series(predictions_vld_0)
rmse_0 = mean_squared_error(target_vld_0, predictions_vld_0)**0.5
mae_0 = mean_absolute_error(predictions_vld_0, target_vld_0)
scores_0 = cross_val_score(model_0, features_vld_0, target_vld_0, cv=5)
final_score_0 = sum(scores_0) / len(scores_0)

print('Cредний запас предсказанного сырья по региону №0: {} тыс. баррелей'.format(round(predictions_vld_0.mean(), 2)))
print('RMSE модели линейной регрессии на валидационной выборке по региону №0: {}'.format(round(rmse_0, 2)))
print('MAE модели линейной регрессии на валидационной выборке по региону №0: {}'.format(round(mae_0, 2)))
print('Усредненный R2 score модели по региону №0: {}'.format(round(final_score_0, 2)))

Cредний запас предсказанного сырья по региону №0: 92.59 тыс. баррелей
RMSE модели линейной регрессии на валидационной выборке по региону №0: 37.58
MAE модели линейной регрессии на валидационной выборке по региону №0: 30.92
Усредненный R2 score модели по региону №0: 0.28


In [10]:
region_data = {
    'region_name': '0',
    'средний_запас_сырья': round(predictions_vld_0.mean(), 2),
    'rmse': round(rmse_0, 2),
    'mae': round(mae_0, 2),
    'r2_score': round(final_score_0, 2)
}
region = pd.DataFrame(region_data, index=[0])
top_regions = region
top_regions

,region_name,средний_запас_сырья,rmse,mae,r2_score
0,0,92.59,37.58,30.92,0.28


In [11]:
model_1 = LinearRegression()
model_1.fit(features_trn_1, target_trn_1)

predictions_vld_1 = model_1.predict(features_vld_1)
predictions_vld_1 = pd.Series(predictions_vld_1)
rmse_1 = mean_squared_error(target_vld_1, predictions_vld_1)**0.5
mae_1 = mean_absolute_error(predictions_vld_1, target_vld_1)
scores_1 = cross_val_score(model_1, features_vld_1, target_vld_1, cv=5)
final_score_1 = sum(scores_1) / len(scores_1)

print('Cредний запас предсказанного сырья по региону №1: {} тыс. баррелей'.format(round(predictions_vld_1.mean(), 2)))
print('RMSE модели линейной регрессии на валидационной выборке по региону №1: {}'.format(round(rmse_1, 2)))
print('MAE модели линейной регрессии на валидационной выборке по региону №1: {}'.format(round(mae_1, 2)))
print('Усредненный R2 score модели по региону №1: {}'.format(round(final_score_1, 2)))

Cредний запас предсказанного сырья по региону №1: 68.77 тыс. баррелей
RMSE модели линейной регрессии на валидационной выборке по региону №1: 0.89
MAE модели линейной регрессии на валидационной выборке по региону №1: 0.72
Усредненный R2 score модели по региону №1: 1.0


In [12]:
region_data = {
    'region_name': '1',
    'средний_запас_сырья': round(predictions_vld_1.mean(), 2),
    'rmse': round(rmse_1, 2),
    'mae': round(mae_1, 2),
    'r2_score': round(final_score_1, 2)
}
top_regions = top_regions.append(region_data, ignore_index = True)
top_regions

C:\Users\lyser\AppData\Local\Temp\ipykernel_21016\1731998016.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_regions = top_regions.append(region_data, ignore_index = True)


,region_name,средний_запас_сырья,rmse,mae,r2_score
0,0,92.59,37.58,30.92,0.28
1,1,68.77,0.89,0.72,1.00


In [13]:
model_2 = LinearRegression()
model_2.fit(features_trn_2, target_trn_2)

predictions_vld_2 = model_2.predict(features_vld_2)
predictions_vld_2 = pd.Series(predictions_vld_2)
rmse_2 = mean_squared_error(target_vld_2, predictions_vld_2)**0.5
mae_2 = mean_absolute_error(predictions_vld_2, target_vld_2)
scores_2 = cross_val_score(model_2, features_vld_2, target_vld_2, cv=5)
final_score_2 = sum(scores_2) / len(scores_2)

print('Cредний запас предсказанного сырья по региону №2: {} тыс. баррелей'.format(round(predictions_vld_2.mean(), 2)))
print('RMSE модели линейной регрессии на валидационной выборке по региону №2: {}'.format(round(rmse_2, 2)))
print('MAE модели линейной регрессии на валидационной выборке по региону №2: {}'.format(round(mae_2, 2)))
print('Усредненный R2 score модели по региону №2: {}'.format(round(final_score_2, 2)))

Cредний запас предсказанного сырья по региону №2: 95.09 тыс. баррелей
RMSE модели линейной регрессии на валидационной выборке по региону №2: 39.96
MAE модели линейной регрессии на валидационной выборке по региону №2: 32.8
Усредненный R2 score модели по региону №2: 0.2


In [14]:
region_data = {
    'region_name': '2',
    'средний_запас_сырья': round(predictions_vld_2.mean(), 2),
    'rmse': round(rmse_2, 2),
    'mae': round(mae_2, 2),
    'r2_score': round(final_score_2, 2)
}
top_regions = top_regions.append(region_data, ignore_index = True)
top_regions

C:\Users\lyser\AppData\Local\Temp\ipykernel_21016\324422654.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_regions = top_regions.append(region_data, ignore_index = True)


,region_name,средний_запас_сырья,rmse,mae,r2_score
0,0,92.59,37.58,30.92,0.28
1,1,68.77,0.89,0.72,1.00
2,2,95.09,39.96,32.80,0.20


### Вывод

Мы обучили модель и получили, что регион №1 имеет наименьший средний запас сырья (68.77 тыс. баррелей), но при этом лучшие показатели метрик оценивания качества моделей (RMSE 0.89, MAE 0.72, R2 score 1).

## Подготовка к расчёту прибыли

Добавим переменные из условия

In [15]:
# Бюджет на разработку скважин в регионе — 10 млрд рублей.
BUDGET = 10000000000

# При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
ALL_POINTS = 500
BEST_POINTS = 200

# При нынешних ценах один баррель сырья приносит 450 рублей дохода.
# Доход с каждой единицы продукта составляет 450 тыс. рублей,
# поскольку объём указан в тысячах баррелей.
INCOME_PER_1000_BARREL = 450000

Посчитаем порог безубыточности в баррелях

In [16]:
# порог безубыточности в баррелях
BREAKEVEN_THRESHOLD = (BUDGET / BEST_POINTS / INCOME_PER_1000_BARREL)

round(BREAKEVEN_THRESHOLD, 2)

111.11

In [17]:
region_data = {
    'region_name': 'breakeven_threshold',
    'rmse': np.nan,
    'mse': np.nan,
    'средний_запас_сырья': round(BREAKEVEN_THRESHOLD, 2),
    'r2_score': np.nan
}
top_regions = top_regions.append(region_data, ignore_index = True)
top_regions

C:\Users\lyser\AppData\Local\Temp\ipykernel_21016\2954731355.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_regions = top_regions.append(region_data, ignore_index = True)


,region_name,средний_запас_сырья,rmse,mae,r2_score,mse
0,0,92.59,37.58,30.92,0.28,NaN
1,1,68.77,0.89,0.72,1.00,NaN
2,2,95.09,39.96,32.80,0.20,NaN
3,breakeven_threshold,111.11,NaN,NaN,NaN,NaN


### Вывод

Порог безубыточности составляет 111.11 тыс. баррелей, во всех наших регионах средний запас сырья ниже этого порога

## Расчёт прибыли и рисков 

### Напишем функцию расчета прибыли для региона

In [18]:
def income(target_vld, predictions_vld):
    sort_series = pd.Series(predictions_vld).sort_values(ascending=False)[:BEST_POINTS]
    target_vld_srt = (target_vld.reset_index(drop = True)[sort_series.index])
    target_sum = target_vld_srt.sum()
    return ((target_sum * INCOME_PER_1000_BARREL) - BUDGET).round(2)

In [19]:
print('Прибыль для региона №0: {} руб.'.format(income(target_vld_0, predictions_vld_0)))

Прибыль для региона №0: 3320826043.14 руб.


In [20]:
income_data = {
    'region_name': '0',
    'прибыль': income(target_vld_0, predictions_vld_0)
}
region = pd.DataFrame(income_data, index=[0])
income_regions = region
income_regions

,region_name,прибыль
0,0,3.320826e+09


In [21]:
print('Прибыль для региона №1: {} руб.'.format(income(target_vld_1, predictions_vld_1)))

Прибыль для региона №1: 2415086696.68 руб.


In [22]:
income_data = {
    'region_name': '1',
    'прибыль': income(target_vld_1, predictions_vld_1)
}
income_regions = income_regions.append(income_data, ignore_index = True)
income_regions

C:\Users\lyser\AppData\Local\Temp\ipykernel_21016\2732641352.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  income_regions = income_regions.append(income_data, ignore_index = True)


,region_name,прибыль
0,0,3.320826e+09
1,1,2.415087e+09


In [23]:
print('Прибыль для региона №2: {} руб.'.format(income(target_vld_2, predictions_vld_2)))

Прибыль для региона №2: 2539915945.84 руб.


In [24]:
income_data = {
    'region_name': '2',
    'прибыль': income(target_vld_2, predictions_vld_2)
}
income_regions = income_regions.append(income_data, ignore_index = True)
income_regions

C:\Users\lyser\AppData\Local\Temp\ipykernel_21016\3344882445.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  income_regions = income_regions.append(income_data, ignore_index = True)


,region_name,прибыль
0,0,3.320826e+09
1,1,2.415087e+09
2,2,2.539916e+09


#### Вывод

Лучший показатель прибыли показал регион №0 (3320826043.14 руб.), худший показатель - регион №1 (2415086696.68 руб.)

### Напишем функцию распределения прибыли, применив технику Bootstrap с 1000 выборок

In [25]:
def bootstrap(target_vld, predictions_vld):
    values = []
    state = np.random.RandomState(12345)
    for i in range(1000):
        srt_series = pd.Series(predictions_vld)
        value = srt_series.sample(n = ALL_POINTS, replace=True, random_state=state)
        values.append(income(target_vld, value))
    values = pd.Series(values)
    mean_income = values.mean()
    print('Средняя прибыль для региона равна {}'.format(round(mean_income,2)))
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    print('95%-й доверительный интервал равен от {} до {}'.format(round(lower,2), round(upper,2)))
    loss_probability = st.percentileofscore(values, 0)
    print('Вероятность убытков для региона равна {}%'.format(round(loss_probability,2)))

In [26]:
bootstrap(target_vld_0, predictions_vld_0)

Средняя прибыль для региона равна 396164984.8
95%-й доверительный интервал равен от -111215545.89 до 909766941.55
Вероятность убытков для региона равна 6.9%


In [27]:
bootstrap(target_vld_1, predictions_vld_1)

Средняя прибыль для региона равна 445617552.56
95%-й доверительный интервал равен от 54585923.7 до 833068150.43
Вероятность убытков для региона равна 1.1%


In [28]:
bootstrap(target_vld_2, predictions_vld_2)

Средняя прибыль для региона равна 340026904.17
95%-й доверительный интервал равен от -208029779.78 до 868855925.84
Вероятность убытков для региона равна 10.7%


In [29]:
income_regions = income_regions.assign(средняя_прибыль = [396164984.8, 445617552.56, 340026904.17])
income_regions = income_regions.assign(вероятность_убытков = [6.9, 1.1, 10.7])
income_regions

,region_name,прибыль,средняя_прибыль,вероятность_убытков
0,0,3.320826e+09,3.961650e+08,6.9
1,1,2.415087e+09,4.456176e+08,1.1
2,2,2.539916e+09,3.400269e+08,10.7


#### Вывод

По условию вероятность убытков должна быть ниже 2.5%, этому соответствует только регион №1 (1.1%), он же показал максимальную среднюю прибыль.  
Стоит отметить, что только у региона №1 95%-й доверительный интервал прибыли полностью положительный.  Это значит, что только в нем новые скважены гарантированно принесут прибыль.

## Общий вывод

Мы изучили начальные данные и определили, что они достаточно качественные: без дубликатов и пропусков. Заметна корреляция (Пирсона), которая говорит о наличии линейной связи между значениями переменных f2 и product. В остальном, достаточно трудно сделать какие-то выводы не имея достаточной информации о переменных.

По условию, для обучения необходимо использовать модель Линейной регрессии.

Мы обучили модель и получили, что регион №1 имеет наименьший средний запас сырья (68.77 тыс. баррелей), но при этом лучшие показатели метрик оценивания качества моделей (RMSE 0.89, MSE 0.72, R2 score 1).

Порог безубыточности составляет 111.11 тыс. баррелей, во всех наших регионах средний запас сырья ниже этого порога

Лучший показатель прибыли показал регион №0 (3320826043.14 руб.), худший показатель - регион №1 (2415086696.68 руб.)

По условию вероятность убытков должна быть ниже 2.5%, этому соответствует только регион №1 (1.1%), он же показал максимальную среднюю прибыль.  

Стоит отметить, что только у региона №1 95%-й доверительный интервал прибыли полностью положительный.  Это значит, что только в нем новые скважены гарантированно принесут прибыль.

Результат исследования: лучший регион для бурения новых скважин - №1.